# Марковские цепи своими руками
TextGen with Markov chains and weirdly compatible datasets

## подготовительный этап

сначала нам нужно задать некоторый текст, на основе которого мы будем генерировать новые предложения.<br>
Создадим документ, в который добавим текст (например, о приготовлении кофе)

In [5]:
import random
import nltk

from nltk import word_tokenize

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


    nltk.download("punkt", quiet=True)


In [7]:
text = """ИНГРЕДИЕНТОВ НА 6ПОРЦИЙ:
яичные желтки 2шт.
сахар 140г.
сливочное масло 50г.
молоко 125мл.
мука 180г.
разрыхлитель 4г.
соль 1 щепотка
яблоки 4шт.

ШАГ 1/8: Желтки взбить с сахаром до посветления желтков и растворения сахара (3 ст. л. оставить для посыпки пирога).

ШАГ 2/8: Растопить сливочное масло. Влить половину масла в яичную массу, добавить молоко, щепотку соли, просеянные муку и разрыхлитель. Все перемешать.

ШАГ 3/8: До однородной массы, без комочков. Перелить тесто в форму для выпечки (у меня 22 см в диаметре), застеленную пекарской бумагой.

ШАГ 4/8: Яблоки очистить от кожуры, удалить сердцевинку, разрезать на четвертинки и затем еще на дольки.

ШАГ 5/8: В тесто выложить дольки яблок вертикально, утапливая их немного в тесте. В произвольном порядке, но чем больше яблок у вас пойдет в пирог, тем вкуснее! :)

ШАГ 6/8: Сверху полить оставшимся маслом (25 г).

ШАГ 7/8: Присыпать оставшимся сахаром. Поставить в разогретую до 180°C духовку на 30-35 минут. Следить за готовностью пирога по своей духовке.

ШАГ 8/8: Готовый пирог остудить и подавать! Приятного! """


with open("MarkovText.txt", "w", encoding="utf-8") as file:
    file.write(text)

    
path = "../data/MarkovText.txt"

In [9]:
len(word_tokenize(text))

214

## часть 1

напишем функцию, которая генерирует новые последовательности слов из заданнного текста

In [14]:
def markov_text(input_text):
    """ This func opens a file, reads its contents and 
    creates a dictionary of words in the markov_gen variable
    based on the number of words a user wants to generate.
    """
    
    data_sample = input_text

    text_data = open(data_sample, 'r').read()
    # text_data = ''.join([i for i in text_data]) 
    text_data = word_tokenize(text_data) # создали лист,  где элементы --это слова
    
    index = 1 # задали первый индекс
    markov_gen = {} # пустой словарь, в него положим ключи(слова) и значения (слова, рядом с ними)
    word_count = int(input('select the number of words you want to generate ')) 
    # задали кол-во итераций: сколько слов будет в итоговом тексте

    for character in text_data[index:]: 
        # для каждого в списке слов датасета, начиная со второго слова
        
        key = text_data[index-1] # ключ - это предыдущее слово
        if key in markov_gen:
            markov_gen[key].append(character)
            # если слово уже в ключах, добавляем к нему в лист значений новое
        else:
            markov_gen[key] = [character] # в ином создаем лист для значений
        index += 1

    character1 = random.choice(list(markov_gen.keys()))
    # превращаем ключи в лист, рандомчойсом выбираем начало
    message = character1.capitalize()

    while len(message.split(' ')) < word_count: # пока слов меньше лимита итераций
        character2 = random.choice(markov_gen[character1]) 
        # рандомно выбираем слово из значений, ключ - предыдущее слово
        character1 = character2 
        message += ' ' + character2
        
    return   message

теперь можно применить функцию к текстовому документу

In [16]:
markov_text(path)

select the number of words you want to generate  90


'От кожуры , тем вкуснее ! : Желтки взбить с сахаром до посветления желтков и затем еще на дольки . сливочное масло 50г . ШАГ 4/8 : Присыпать оставшимся маслом ( у вас пойдет в пирог , утапливая их немного в форму для выпечки ( у вас пойдет в диаметре ) . молоко 125мл . ШАГ 4/8 : Растопить сливочное масло 50г . ШАГ 6/8 : Яблоки очистить от кожуры , просеянные муку и растворения сахара ( 3 ст . Перелить тесто в форму для посыпки пирога ) ШАГ 8/8'

Мы сгенерировали новое предложение длиной в n слов,о том как готовить яблочный пирог. *(пока что они выглядят странно, это нормально)*

## часть 2 : попробуем генерировать истории


In [17]:
! pip3 install markovify # установим библиотеку для марковских цепей

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 235 kB 1.5 MB/s            
Using legacy 'setup.py install' for markovify, since package 'wheel' is not installed.
    Running setup.py install for markovify ... done


In [18]:
import markovify 

попробуем [на текстах басен Крылова](https://github.com/sjut/DPO_Materials/blob/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/basni.zip):

In [19]:
# сохраним текстовый файл, откроем его

with open("../data/basni.txt", "r",encoding = "utf-8") as file:
    data = file.read()

In [20]:
 # display(data) # здесь можно посмотреть на содержимое файла

'«Как, милый Петушок, поешь, ты громко, важно!»-\n«А ты, Кукушечка, мой свет,\nКак тянешь плавно и протяжно:\nВо всем лесу у нас такой певицы нет!» —\n«Тебя, мой куманек, век слушать я готова».—\n«А ты, красавица, божусь,\nЛишь только замолчишь, то жду я, не дождусь,\nЧтоб начала ты снова…\nОтколь такой берется голосок?\nИ чист, и нежен, и высок!..\nДа вы уж родом так: собою невелички,\nА песни, что твой соловей!» —\n«Спасибо, кум; зато, по совести моей,\nПоешь ты лучше райской птички,\nНа всех ссылаюсь в этом я».\nТут Воробей, случась, примолвил им: «Друзья!\nХоть вы охрипните, хваля друг дружку,—\nВсе ваша музыка плоха!..»\nБасня Квартет читать\nПроказница-Мартышка, Осел, Козел да косолапый Мишка\nЗатеяли сыграть Квартет.\nДостали нот, баса, альта, две скрипки\nИ сели на лужок под липки -\nПленять своим искусством свет.\nУдарили в смычки, дерут, а толку нет.\n"Стой, братцы, стой! - кричит Мартышка. - Погодите!\nКак музыке идти? Ведь вы не так сидите.\nТы с басом, Мишенька, садись про

In [32]:
initial_text = markovify.Text(data, state_size =1) 
# последний аргумент регулирует размер повторяющихся цепочек

for i in range(4): # задаем количество предложений 
    print(initial_text.make_sentence(), sep = "\n")

и — говорит, — говорит, — «Спасибо, кум; зато, по роще рассыпался.
Случилось Соловью на лад нейдет.
Осел, уставясь в смычки, дерут, а толку нет.
Случилось Соловью на приветливы Лисицыны слова Ворона каркнула во рту держала.


Мы можем задавать и количество слов в предложении, и количество предложений

In [31]:
for i in range(4): # кол-во предложений
    print(initial_text.make_short_sentence(70), sep = "\n") # кол-во слов в предложении

Ведь вы уж собралась, Да призадумалась, а толку нет.
Как тянешь плавно и как хороша!
Ударили в пещере, как сидеть.
Поверишь ли, в смычки, дерут, а толку нет.


## часть 3: что делать дальше

Попробуйте сгенерировать интересные тексты из материалов, которые интересны Вам: художественная литература, новости, статьи, и  тд. Как правило, чем больше изначального текста, тем больше вероятность, что генерированный текст будет выглядеть естественно

In [58]:
with open("../data/corgi.txt", "r") as corgi, open("../data/d_speech.txt", "r") as chekhov:
    corgi = markovify.Text(corgi.read(),well_formed=True,state_size =1) 
    chekhov = markovify.Text(chekhov.read(),well_formed=True,state_size =1)

In [70]:
for i in range(2): # кол-во предложений
    print(corgi.make_sentence(),
         chekhov.make_sentence(), sep = "\n") # кол-во слов в предложении

Герцог Йоркский выбрал именно корги могут возникать проблемы со здоровьем.
И неужели прислуге не замечает своего опасного состояния, или без пашота?
По обучаемости эта порода противопоказана.
и ещё рады, что за пределы живота...
